![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## AI Inference on a video stream
This notebook is a simple example of how to use DeGirum PySDK to do AI inference on a video stream.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

You also need to specify your cloud API access token in `degirum_cloud_token`.

You can change `video_source` to index of a local webcamera, or URL of an RTSP stream, or URL of a YouTube video, or path to another video file.


In [ ]:
# make sure degirum-tools package is installed
!pip show degirum || pip install degirum

#### Specify where you want to run your inferences, model zoo url, model name and video source

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local web camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# degirum_cloud_token: your token for accessing the DeGirum cloud platform
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
video_source = (
    "https://github.com/DeGirum/PySDKExamples/raw/main/images/example_video.mp4"
)
degirum_cloud_token= "<enter your token here>"

In [ ]:
# Import the necessary libraries
import degirum as dg
import cv2

# Connect to the AI inference engine
# hw_location: Hardware location
# model_zoo_url: URL of the model zoo
# degirum_cloud_token: Token for the Degirum cloud
zoo = dg.connect(hw_location, model_zoo_url, degirum_cloud_token)

# Load the object detection AI model from the model zoo
# model_name: Name of the model to load
# overlay_show_probabilities: Whether to show probabilities in the overlay
model = zoo.load_model(model_name, overlay_show_probabilities=True)

# Open the video source
# video_source: Path to the video file or ID of the camera
cap = cv2.VideoCapture(video_source)

# Use the model as a context manager to ensure resources are properly managed
with model as ml_model:
    # Start an infinite loop to process the video frame by frame
    while True:
        # Read a frame from the video stream
        ret, frame = cap.read()
        # If the frame was not read successfully, break the loop
        if not ret:
            break
        # Process the frame using the model
        # The result is an object that includes the processed frame and other information
        inference_result = ml_model(frame)
        # Display the processed frame with an overlay in a window named 'Image'
        cv2.imshow('Image', inference_result.image_overlay)
        # Wait for 1 millisecond and check if the 'q' key was pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            # If 'q' was pressed, destroy all windows and break the loop
            cv2.destroyAllWindows()
            break

# After the loop ends, release the video capture object to free up resources
cap.release()
# Destroy any remaining OpenCV windows
cv2.destroyAllWindows()